# DEEPFACE COMPLET WORK

In [ ]:
import cv2
import gc
import pandas as pd
from deepface import DeepFace

def analyzer(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return result
    return wrapper

@analyzer    
def emotionDetect(img):
    return DeepFace.analyze(img, ["emotion"], detector_backend='ssd', enforce_detection=False)

@analyzer
def faceDetect(img, path):
    return DeepFace.verify(img, path, distance_metric="euclidean_l2",
                           enforce_detection=False, detector_backend='ssd')

time_target = 5
indice = 0

source = cv2.VideoCapture(0)

while True:
    ret, frame = source.read()
    frame = cv2.flip(frame, 1)
    frame = cv2.resize(frame, (640, 480))
    if not ret:
        break

    detector = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector.detectMultiScale(gray, scaleFactor=1.05, minNeighbors=6, minSize=(100, 100))

    for x, y, w, h in faces:
        if indice % time_target == 0:
            result2 = faceDetect(frame, 'Face Recognition/Yassine1.jpg')

        cv2.rectangle(frame, (x, y), (x + w, y - int(frame.shape[1] / 40)),
                      (200, 200, 200), cv2.FILLED)
        if indice % time_target == 0:
            result1 = emotionDetect(frame)
            res1 = result1[0]["dominant_emotion"]

        if result2['verified']:
            cv2.rectangle(frame, (x, y), (x + w, y + h),
                          (50, 250, 50), thickness=int(frame.shape[0] * 0.005), lineType=cv2.LINE_AA)
            cv2.putText(frame, str('Yassine is ' + res1), (x, y - 5), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0))
        else:
            cv2.rectangle(frame, (x, y), (x + w, y + h),
                          (50, 50, 250), thickness=int(frame.shape[0] * 0.005), lineType=cv2.LINE_AA)
            cv2.putText(frame, str('someone is ' + res1), (x, y - 5), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0))
            indice += 1

            
    cv2.imshow('Live', frame)
    gc.collect()
    if cv2.waitKey(int(1000/30)) & 0xFF == ord('q'):
        break

source.release()
cv2.destroyAllWindows()

